In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Conv2DTranspose, AveragePooling2D, Add
from scipy.interpolate import RectBivariateSpline
import tensorflow.contrib.eager as tfe
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Boundary import Boundary1D
from collections.abc import Iterable
import itertools, h5py
from multiprocessing import Pool as ThreadPool
opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.95)
conf = tf.ConfigProto(gpu_options=opts)
tfe.enable_eager_execution(config=conf)

In [ ]:
class Upsample(tf.keras.layers.Layer):
    def __init__(self, upsample_ratio = (2,2), output_dtype = tf.float32, data_format = 'channels_first', resize_method = tf.image.ResizeMethod.BICUBIC, **kwargs):
        super(Upsample, self).__init__(**kwargs)
        self.output_dtype = output_dtype
        self.data_format = data_format
        self.resize_method = resize_method
        if not isinstance(upsample_ratio, Iterable):
            self.upsample_ratio = (upsample_ratio, upsample_ratio)
        else:
            self.upsample_ratio = upsample_ratio
            
    def build(self, input_shape):
        super(Upsample, self).build(input_shape)
        
    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            return tf.TensorShape(list(input_shape)[0:2] + [self.upsample_ratio[0] * input_shape[-2], self.upsample_ratio[1] * input_shape[-1]])
        else:
            return tf.TensorShape([input_shape[0], self.upsample_ratio[0] * input_shape[-3], self.upsample_ratio[1] * input_shape[-2], input_shape[-1]])
    
#     def upsample(self, inp):
#         dummy_coords_inp = (np.linspace(0,1,int(inp.shape[-2])), np.linspace(0,1,int(inp.shape[-1])))
#         dummy_coords_out = (np.linspace(0,1,int(self.compute_output_shape(inp.shape)[-2])),np.linspace(0,1,int(self.compute_output_shape(inp.shape)[-1])))
#         #pdb.set_trace()
#         res = []
#         for i in range(inp.shape[0]):
#             spl = RectBivariateSpline(dummy_coords_inp[0], dummy_coords_inp[1], np.array(inp[i,...]))
#             res.append(spl(dummy_coords_out[0], dummy_coords_out[1]))
#         return tf.dtypes.cast(np.array(res), dtype=self.output_dtype)
    
#     @tf.contrib.eager.defun
#     def upsample_wrap(self,inp):
#         return tf.map_fn(self.upsample, inp)
        
    def call(self, inputs):
#         return self.upsample_wrap(inputs)
        if self.data_format == 'channels_first':
            return tf.transpose(tf.image.resize_images(tf.transpose(inputs, (0,2,3,1)), tf.multiply(self.upsample_ratio, inputs.shape[2:]), align_corners=True, method=self.resize_method), (0,3,1,2))
        else:
            return tf.image.resize_images(inputs, tf.multiply(self.upsample_ratio, inputs.shape[1:-1]), align_corners=True, method=self.resize_method)
        

In [ ]:
def poisson_matrix(m,n):
    '''
    Generates the matrix A to express the Poisson equation in the form Ax=b for an m-by-n grid
    
    Them matrix returned shall be (m-2)*(n-2)-by-(m-2)*(n-2) in size
    
    CURRENTLY ONLY WORKS FOR SQUARE DOMAINS!!!!
    '''
    m = m-2
    n = n-2
    
    D = np.zeros((m,m), dtype = np.float64)
    i,j = np.indices(D.shape)
    D[i==j] = 4.0
    D[i==j-1] = -1.0
    D[i==j+1] = -1.0
    
    S = -np.eye(D.shape[0], dtype = np.float64)
    
    P = np.zeros((m*n,m*n), dtype = np.float64)
    ind = np.arange(0,m*(n+1), m)
    
    for i in range(len(ind)-1):
        P[ind[i]:ind[i+1], ind[i]:ind[i+1]] = D
        try:
            P[ind[i+1]:ind[i+2], ind[i]:ind[i+1]] = S
        except:
            pass
        try:
            P[ind[i-1]:ind[i], ind[i]:ind[i+1]] = S
        except:
            pass
    return P

def generate_random_RHS(n, n_controlpts = None, n_outputpts = None, s = 5, domain = [0,1,0,1]):
    
    '''
    This function generates random smooth RHS 'functions' defined pointwise using bivariate splines. 
    n: no. of random RHSes to generate
    n_controlpts: no. of control pts of the spline. Smaller values lead to 'smoother' results
    n_outputpts: no. of gridpoints in each direction of the output 
    s: see parameter s in scipy.interpolate.RectBivariateSpline
    domain: [x_min, x_max, y_min, y_max]
    '''
    
    
    if isinstance(n, Iterable):
        n_controlpts = n[1]
        n_outputpts = n[2]
        try:
            s = n[3]
        except:
            pass
        try:
            domain = n[4]
        except:
            pass
        n = n[0]
    
    x = np.linspace(domain[0], domain[1], n_controlpts)
    y = np.linspace(domain[2], domain[3], n_controlpts)
    if n_controlpts != n_outputpts:
        x_out = np.linspace(domain[0], domain[1], n_outputpts)
        y_out = np.linspace(domain[2], domain[3], n_outputpts)
    else:
        x_out = x
        y_out = y
            
    out = []
    for i in range(n):
        spl = RectBivariateSpline(x,y,2*np.random.rand(len(x), len(y))-1, s=s)
        out.append(spl(x_out,y_out))
    return np.array(out)
    

def poisson_RHS(F, boundaries = None, h = None):
    '''
    Generates the RHS vector b of a discretized Poisson problem in the form Ax=b.
    h = grid spacing
    boundaries = dict containing entries 'top', 'bottom', 'right' and 'left' which correspond to the Dirichlet BCs at these boundaries. Each entry must be a vector of length m or n, where m and n are defined as in te function poisson_matrix
    F = an m by n matrix containing the RHS values of the Poisson equation
    
    (i.e. this function merely takes the BC information and the array from generate_random_RHS to provide the RHS for the matrix eq. form)
    '''
    
    if isinstance(F, Iterable):
        boundaries = F[1]
        h = F[2]
        F = F[0]
    
    F = -h**2 * F
    F[...,1:-1,1] = F[...,1:-1,1] + np.array(boundaries['top'])[1:-1]
    F[...,1:-1,-2] = F[...,1:-1,-2] + np.array(boundaries['bottom'])[1:-1]
    F[...,1,1:-1] = F[...,1,1:-1] + np.array(boundaries['left'])[1:-1]
    F[...,-2,1:-1] = F[...,-2,1:-1] + np.array(boundaries['right'])[1:-1]
    
    return F[...,1:-1,1:-1].reshape(list(F[...,1:-1,1:-1].shape[:-2]) + [np.prod(F[...,1:-1,1:-1].shape[-2:])])
 
def generate_dataset(batch_size, n, h, boundaries, n_batches = 1, rhs_range = [-1,1]):
    lhs = tf.constant(poisson_matrix(n,n), dtype=tf.float64)
    lhs_chol = tf.linalg.cholesky(lhs)
    
    def chol(r):
        return tf.linalg.cholesky_solve(lhs_chol, tf.transpose(tf.stack([r])))
    
    @tf.contrib.eager.defun
    def chol_solve(rhs_arr):
        return tf.map_fn(chol, rhs)
    
    #pdb.set_trace()
    pool = ThreadPool(n_batches)
    F = pool.map(generate_random_RHS, zip(itertools.repeat(batch_size, n_batches), itertools.repeat(10), itertools.repeat(n), itertools.repeat(5), itertools.repeat([0,n*h,0,n*h])))
    rhs = tf.concat(pool.map(poisson_RHS, zip(F, itertools.repeat(boundaries), itertools.repeat(h))), axis=0)
    
    soln = np.zeros((n_batches * batch_size, n, n), dtype = np.float64)
    soln[...,:,0] = boundaries['top']
    soln[...,:,-1] = boundaries['bottom']
    soln[...,0,:] = boundaries['left']
    soln[...,-1,:] = boundaries['right']
    soln[:,1:-1,1:-1] = tf.reshape(chol_solve(rhs), (n_batches * batch_size, n-2, n-2))
    #soln = chol_solve(rhs)
    
    #return tf.reshape(soln, (n_batches, batch_size, lhs_chol.shape[0])), tf.reshape(F, (n_batches, batch_size, F.shape[-2], F.shape[-1]))
    return tf.reshape(soln, (n_batches*batch_size, 1, soln.shape[-2], soln.shape[-1])), tf.reshape(tf.concat(F, axis = 0), (n_batches*batch_size, 1, F[0].shape[-2], F[0].shape[-1]))


In [102]:
def Lp_integrate_batch(inp):
    #unpack values
    data = tf.transpose(inp[0], (2,3,1,0))
    b = inp[1]
    w = inp[2]
    ind = inp[3]
    p = inp[4]
    #print(w)
    #get the points from the image to perform interpolation on
    interp_pts = tf.squeeze(tf.gather_nd(data, ind))
    #pdb.set_trace()
    #multiply image values with the weights b ti interpolate original image onto the GL quadrature points
    values_at_quad_pts = tf.einsum('ijkl, ijk->ijl', interp_pts, b)
    #pdb.set_trace()
    #compute Lp norm
    return tf.pow(tf.reduce_sum(tf.einsum('ij,ijk->ijk',w,tf.pow(values_at_quad_pts, p)), axis = (0,1)), 1/p)
    

def Lp_integral_norm(image_size, domain, n_quadpts = 10, quadpts_randomization = 5, p=2):
    interpolation_weights = []
    quadweights_list = []
    index_combinations_list = []
    coords = np.array(np.meshgrid(np.linspace(domain[0], domain[1], image_size[0]),np.linspace(domain[2], domain[3], image_size[1]),indexing = 'xy'), dtype = np.float32).transpose((1,2,0))
    c = np.array([np.array(0.5*(domain[1] - domain[0]),dtype=np.float64),np.array(0.5*(domain[3] - domain[2]),dtype=np.float32)]) #scaling coefficients - for handling domains other than [-1,1] x [-1,1]
    d = np.array([np.array(0.5*(domain[1] + domain[0]),dtype=np.float64),np.array(0.5*(domain[3] + domain[2]),dtype=np.float32)])
    for n in range(n_quadpts - quadpts_randomization, n_quadpts + quadpts_randomization+1):
        quadrature_x, quadrature_w = tuple([np.polynomial.legendre.leggauss(n)[i].astype(np.float64) for i in range(2)]) #quadrature weights and points
        quadpts = tf.constant(np.apply_along_axis(lambda x: x + d, 0, np.einsum('ijk,i->ijk',np.array(np.meshgrid(quadrature_x,quadrature_x,indexing = 'xy')),c)).transpose((1,2,0)),dtype = tf.float32)
        quadweights = tf.reduce_prod(c)*tf.tensordot(quadrature_w,quadrature_w,axes = 0)
        indices = [[],[]]
        quad_coords = tf.stack([quadpts[0,:,0], quadpts[:,1,1]])
        image_coords = tf.stack([coords[0,:,0], coords[:,1,1]])
        for i in range(len(indices)):
            j=0
            while len(indices[i]) < quadpts.shape[0] and j<image_coords[i].shape[0]:
                #pdb.set_trace()
                #print(j)
                if abs(float(quad_coords[i,len(indices[i])] - image_coords[i,j])) == float(min(abs(quad_coords[i,len(indices[i])] - image_coords[i,j-1]), abs(quad_coords[i,len(indices[i])] - image_coords[i,j]), abs(quad_coords[i,len(indices[i])] - image_coords[i,j+1]))):
                    if quad_coords[i,len(indices[i])] - image_coords[i,j] < 0:
                        indices[i].append((j-1,j))
                    else:
                        indices[i].append((j,j+1))
                j+=1
        

        quadweights_list.append(quadweights)
        index_combinations = np.zeros((quadpts.shape[0], quadpts.shape[1], 4 , 2), dtype = np.int32)
        corners = np.zeros((quadpts.shape[0], quadpts.shape[1], 2 , 2), dtype = np.int32)
        #corner_coords = np.zeros(corners.shape)
        s=np.array(indices)
        for i in range(n):
            for j in range(n):
                index_combinations[i,j,:,:] = np.array(list(itertools.product(np.array(s)[0,i,:],np.array(s)[1,j,:])))
        for i in range(n):
            for j in range(n):
                corners[i,j,:,:] = np.array([s[0,i,:],s[1,j,:]])
        corners = corners.transpose((0,1,3,2))
#         for i in range(n):
#             for j in range(n):
#                 for k in range(2):
#                     corner_coords[i,j,k,:] = coords[corners[i,j,k,0],corners[i,j,k,1],:]
        corner_coords = tf.transpose(tf.gather_nd(coords,corners),(1,0,2,3))
        interpolation_matrix = np.ones((n,n,4,4))
        interpolation_matrix[:,:,0:2,1] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,0:2,1],corner_coords[:,:,0,0])
        interpolation_matrix[:,:,2:,1] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,2:,1],corner_coords[:,:,1,0])
        interpolation_matrix[:,:,(0,2),2] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,(0,2),2],corner_coords[:,:,0,1])
        interpolation_matrix[:,:,(1,3),2] = np.einsum('ijk,ij->ijk',interpolation_matrix[:,:,(1,3),2], corner_coords[:,:,1,1])
        interpolation_matrix[:,:,:,3] *= np.multiply(interpolation_matrix[:,:,:,1], interpolation_matrix[:,:,:,2])
        #pdb.set_trace()
        interpolation_matrix = tf.transpose(tf.linalg.inv(interpolation_matrix), (0,1,3,2))
        q = np.ones((n,n,4))
        q[:,:,1] = tf.transpose(quadpts[:,:,0])
        q[:,:,2] = tf.transpose(quadpts[:,:,1])
        q[:,:,3] = np.multiply(q[:,:,1],q[:,:,2])
        
        b = tf.einsum('ijkl, ijl->ijk', tf.constant(interpolation_matrix), tf.constant(q))
        #pdb.set_trace()
        index_combinations_list.append(index_combinations)
        interpolation_weights.append(b)
        #return interpolation_matrix[2,4,:,:], q[2,4,:], b[2,4,:]
    
    #pdb.set_trace()
    
#     def Lp_integrate_batch(inp):
#         #unpack values
#         data = tf.transpose(inp[0], (2,3,1,0))
#         b = inp[1]
#         w = inp[2]
#         ind = inp[3]
#         p = inp[4]
        
#         #get the points from the image to perform interpolation on
#         interp_pts = tf.squeeze(tf.gather_nd(data, ind))
#         #multiply image values with the weights b ti interpolate original image onto the GL quadrature points
#         values_at_quad_pts = tf.einsum('ijkl, ijk->ijl', interp_pts, b)
#         #compute Lp norm
#         norm = tf.pow(tf.reduce_sum(tf.einsum('ij,ijk->ijk',w,tf.pow(values_at_quad_pts, p)), axis = (0,1)), 1/p)
    
    #@tfe.defun
    def Lp_integrate(y_true,y_pred, b=interpolation_weights, w=quadweights_list, ind=index_combinations_list, p=p):
        #pool = ThreadPool(len(b))
        return tf.concat(list(map(Lp_integrate_batch, zip(tf.split(y_true-y_pred, len(b)), itertools.cycle(b), itertools.cycle(w), itertools.cycle(ind), itertools.repeat(p)))), 0)
        
    return Lp_integrate
    

In [ ]:
ntest = 64
h = 0.05
boundary_top = Boundary1D('Dirichlet', [(0,ntest*h),(ntest*h,ntest*h)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)
boundary_right = Boundary1D('Dirichlet', [(ntest*h,ntest*h),(ntest*h,0)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)
boundary_bottom = Boundary1D('Dirichlet', [(ntest*h,0),(0,0)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)
boundary_left = Boundary1D('Dirichlet', [(0,0),(0,ntest*h)], orientation='clockwise', RHS_function=lambda t: t-t, boundary_rhs_is_parametric=True)

In [ ]:
import time
t0 = time.time()
soln,F = generate_dataset(batch_size=2000, n = ntest, h = h, n_batches=20, boundaries={'top': boundary_top.RHS_evaluate(np.linspace(boundary_top.t.min(),boundary_top.t.max(),ntest)), 'right': boundary_right.RHS_evaluate(np.linspace(boundary_right.t.min(),boundary_right.t.max(),ntest)), 'bottom': boundary_bottom.RHS_evaluate(np.linspace(boundary_bottom.t.min(),boundary_bottom.t.max(),ntest)), 'left': boundary_left.RHS_evaluate(np.linspace(boundary_left.t.min(),boundary_left.t.max(),ntest))})
t1 = time.time()
print('Generation of training data took ' + str(t1-t0) + ' seconds')
with h5py.File('dataset8.h5', 'w') as hf:
    hf.create_dataset('soln', data=soln)
    hf.create_dataset('F', data=F)

In [ ]:
for i in range(6,9):
    with h5py.File('dataset' + str(i) + '.h5', 'r') as hf:
        F = np.array(hf.get('F'), dtype = np.float64)
        soln = np.array(hf.get('soln'), dtype = np.float64)
        try:
            train_data = train_data.concatenate(tf.data.Dataset.from_tensor_slices((F,soln)))
        except:
            train_data = tf.data.Dataset.from_tensor_slices((F,soln))

In [ ]:
import time
t0 = time.time()
soln_valid,F_valid = generate_dataset(batch_size=1000, n = ntest, h = h, n_batches=2, boundaries={'top': boundary_top.RHS_evaluate(np.linspace(boundary_top.t.min(),boundary_top.t.max(),ntest)), 'right': boundary_right.RHS_evaluate(np.linspace(boundary_right.t.min(),boundary_right.t.max(),ntest)), 'bottom': boundary_bottom.RHS_evaluate(np.linspace(boundary_bottom.t.min(),boundary_bottom.t.max(),ntest)), 'left': boundary_left.RHS_evaluate(np.linspace(boundary_left.t.min(),boundary_left.t.max(),ntest))})
t1 = time.time()
print('Generation of validation data took ' + str(t1-t0) + ' seconds')

In [ ]:
# with h5py.File('dataset' + str(4) + '.h5', 'r') as hf:
#         F_valid = np.array(hf.get('F'), dtype = np.float64)
#         soln_valid = np.array(hf.get('soln'), dtype = np.float64)

In [ ]:
shuffle_size = 100000
batch_size = 2000
train_data = train_data.shuffle(shuffle_size).batch(batch_size)
dataset_validation = tf.data.Dataset.from_tensor_slices((F_valid, soln_valid)).shuffle(shuffle_size).batch(100)
#del F, soln#, F_valid, soln_valid

In [ ]:
input_0 = Input(shape=(1,ntest,ntest,))
conv_1_0 = Conv2D(filters = 5, kernel_size = 3, activation=tf.nn.relu, data_format='channels_first', padding='same')(input_0)

conv_2_0 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format='channels_first', padding='same')(conv_1_0)
pool_2_0 = AveragePooling2D(data_format = 'channels_first')(conv_2_0)
pool_2_1 = AveragePooling2D(data_format = 'channels_first')(pool_2_0)

conv_3_0 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format = 'channels_first', padding='same')(conv_2_0)
conv_3_1 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format = 'channels_first', padding='same')(pool_2_0)
conv_3_2 = Conv2D(filters = 8, kernel_size = 3, activation=tf.nn.relu, data_format = 'channels_first', padding='same')(pool_2_1)
#transposeconv_3_1 = Conv2DTranspose(filters = 8, kernel_size = 3, strides = 2, activation = tf.nn.relu, data_format = 'channels_first', padding='same')(conv_3_1)
#transposeconv_3_2 = Conv2DTranspose(filters = 8, kernel_size = 3, strides = 4, activation = tf.nn.relu, data_format = 'channels_first', padding='same')(conv_3_2)
#upsample_3_1 = UpSampling2D(size=(2,2), data_format = 'channels_first', interpolation = 'bilinear')(conv_3_1)
#upsample_3_2= UpSampling2D(size=(4,4), data_format = 'channels_first', interpolation = 'bilinear')(conv_3_2)
upsample_3_1 = Upsample()(conv_3_1)
upsample_3_2 = Upsample(4)(conv_3_2)
#merge_3_0 = Add()([conv_3_0, transposeconv_3_1, transposeconv_3_2])
merge_3_0 = Add()([conv_3_0, upsample_3_1, upsample_3_2])

conv_4_0 = Conv2D(filters = 8, kernel_size = 1, activation = tf.nn.relu, data_format = 'channels_first', padding='same')(merge_3_0)

conv_5_0 = Conv2D(filters = 1, kernel_size = 1, activation = 'linear', data_format = 'channels_first', padding='same')(conv_4_0)
final_activation = tf.keras.layers.PReLU()(conv_5_0)
a = Model(input_0, final_activation)

In [ ]:
a.compile(optimizer = tf.train.AdamOptimizer(learning_rate=1e-3), loss = tf.losses.mean_squared_error, metrics = ['accuracy'])
a.summary()

In [ ]:
a.fit(train_data, steps_per_epoch=8, epochs=7, validation_data=dataset_validation, validation_steps=2)

In [ ]:
# a.save_weights('model.h5')

In [ ]:
p = np.random.randint(0,F_valid.shape[0])
y, x = np.meshgrid(np.linspace(0, ntest*h, ntest), np.linspace(0, ntest*h, ntest))
z = a.predict(tf.expand_dims(F[p,...], axis=0))[0,0,...]
#z = soln[p,0,...]
#z = generate_random_RHS(10, n_controlpts=10, n_outputpts=64)[4,:,:]
#z = a.predict(tf.expand_dims(F_valid[p,...], axis=0))[0,0,...] - soln_valid[p,0,...]
z_min, z_max = -np.abs(z).max(), np.abs(z).max()
fig, ax = plt.subplots()
c = ax.pcolormesh(x, y, z, cmap='RdBu', vmin=z_min, vmax=z_max)
ax.set_title('pcolormesh')
ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
#soln_valid,F_valid = generate_dataset(batch_size=50, n = ntest, h = h, n_batches=1, boundaries={'top': boundary_top.RHS_evaluate(np.linspace(boundary_top.t.min(),boundary_top.t.max(),ntest)), 'right': boundary_right.RHS_evaluate(np.linspace(boundary_right.t.min(),boundary_right.t.max(),ntest)), 'bottom': boundary_bottom.RHS_evaluate(np.linspace(boundary_bottom.t.min(),boundary_bottom.t.max(),ntest)), 'left': boundary_left.RHS_evaluate(np.linspace(boundary_left.t.min(),boundary_left.t.max(),ntest))})
#p = np.random.randint(0,F_valid.shape[0])
y, x = np.meshgrid(np.linspace(0, ntest*h, ntest), np.linspace(0, ntest*h, ntest))
z = F[p,0,...]
#z = soln[p,0,...]
z_min, z_max = -np.abs(z).max(), np.abs(z).max()
fig, ax = plt.subplots()
c = ax.pcolormesh(x, y, z, cmap='RdBu', vmin=z_min, vmax=z_max)
ax.set_title('pcolormesh')
ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
i = np.random.randint(1,ntest-2)
j = np.random.randint(1,ntest-2)
p = np.random.randint(0,soln_valid.shape[0]-1)
print((-4 * soln_valid[p,0,i,j] + soln_valid[p,0,i,j-1] + soln_valid[p,0,i-1,j] + soln_valid[p,0,i+1,j] + soln_valid[p,0,i,j+1])/(h**2))
example_soln = a.predict(tf.expand_dims(F_valid[p,...], axis=0))
print((-4 * example_soln[0,0,i,j] + example_soln[0,0,i,j-1] + example_soln[0,0,i-1,j] + example_soln[0,0,i+1,j] + example_soln[0,0,i,j+1])/(h**2))
print(F_valid[p,0,i,j])
print(soln_valid[p,0,i,j])

In [ ]:
X,Y = np.meshgrid(np.linspace(0,1,20),np.linspace(0,1,20))
Z_m = np.array([[np.sin(np.pi*(X+Y)), np.cos(np.pi * (X+Y))] , [np.exp(-X**2 * Y**2), np.tan(np.multiply(X,Y))]])
#Z_m = tf.expand_dims(Z, axis = 0)
ur = 3
b = BilinearUpsample(ur)(input_0)
model1 = Model(input_0, b)

In [ ]:
Z_m.shape

In [ ]:
X0, Y0 = np.meshgrid(np.linspace(0,1,X.shape[0]*ur),np.linspace(0,1,X.shape[0]*ur))
fig, ax = plt.subplots()
c = ax.pcolormesh(X0, Y0, model1(Z_m)[0,0,...], cmap='RdBu')

In [ ]:
fig, ax = plt.subplots()
c = ax.pcolormesh(X, Y, Z_m[0,0,...], cmap='RdBu')

In [ ]:
np.mean(np.abs(a.predict(F_valid) - soln_valid))

In [ ]:
timeit a.predict(tf.expand_dims(F_valid[0,...], axis=0))

In [ ]:
timeit a.predict(F_valid[0:1000,...])

In [ ]:
lhs = poisson_matrix(64,64)
lhs_chol = tf.to_float(tf.linalg.cholesky(lhs))
print(lhs_chol.shape)
rhs = tf.constant(np.random.rand((64-2)**2,1000ur), dtype=tf.float32)
print(rhs.shape)

In [ ]:
timeit tf.linalg.cholesky_solve(lhs_chol,rhs)

In [ ]:
n_quadpts = 12
domain = [0,3.2,0,3.2]
quadrature_x, quadrature_w = tuple([np.polynomial.legendre.leggauss(n_quadpts)[i].astype(np.float32) for i in range(2)]) #quadrature weights and points
c = np.array([np.array(0.5*(domain[1] - domain[0]),dtype=np.float32),np.array(0.5*(domain[3] - domain[2]),dtype=np.float32)]) #scaling coefficients - for handling domains other than [-1,1] x [-1,1]
d = np.array([np.array(0.5*(domain[1] + domain[0]),dtype=np.float32),np.array(0.5*(domain[3] + domain[2]),dtype=np.float32)])
quadpts = tf.constant(np.apply_along_axis(lambda x: x + d, 0, np.einsum('ijk,i->ijk',np.array(np.meshgrid(quadrature_x,quadrature_x,indexing = 'xy')),c)).transpose((1,2,0)),dtype = tf.float32)
quadweights = tf.reduce_prod(c)*tf.tensordot(quadrature_w,quadrature_w,axes = 0)

In [ ]:
print(quadpts[0,:,0])
x_image = np.linspace(domain[0], domain[1], ntest)
print(x_image)

In [ ]:
import pdb
indices = []
j=0
while len(indices) < quadpts.shape[0] and j<x_image.shape[0]:
    #pdb.set_trace()
    #print(j)
    if abs(float(quadpts[0,len(indices),0] - x_image[j])) == float(min(abs(quadpts[0,len(indices),0] - x_image[j-1]), abs(quadpts[0,len(indices),0] - x_image[j]), abs(quadpts[0,len(indices),0] - x_image[j+1]))):
        if quadpts[0,len(indices),0] - x_image[j] < 0:
            indices.append((j-1,j))
        else:
            indices.append((j,j+1))

    j+=1
    

In [ ]:
indices

In [ ]:
tf.stack([np.array([3,4,5]), np.array([0,1,2])]).shape

In [86]:
import pdb
print(Lp_integral_norm((64,64), [0,1,0,2], quadpts_randomization=1))
# s = np.array(L2_integral_norm((64,64), [0,3.2,0,3.2], quadpts_randomization=1)[2][0])
# print(s)
# print(s[0])
# print(len(s[0]))
# print(s[1][2][0])

> <ipython-input-85-bc7d2a1c88d0>(78)Lp_integral_norm()
-> index_combinations_list.append(index_combinations)
[1.         0.33787328 1.83603108 0.62034584]
tf.Tensor([0.11782257 0.5961614  0.04719874 0.23881729], shape=(4,), dtype=float64)


BdbQuit: 

In [ ]:
np.array(list(itertools.product(np.array(s)[0,5,:],np.array(s)[1,3,:])))

In [ ]:
index_combinations = np.zeros((s.shape[1], s.shape[1], 4 , 2), dtype = np.int32)
for i in range(9):
    for j in range(9):
        index_combinations[i,j,:,:] = np.array(list(itertools.product(np.array(s)[0,i,:],np.array(s)[1,j,:])))
#print(index_combinations)
coords = np.array(np.meshgrid(x_image,x_image)).transpose(1,2,0)
print(index_combinations[0,0,0,:])
coords[index_combinations[0,0,1,0],index_combinations[0,0,1,1],:]
testdata 

In [ ]:
testdata = tf.constant(np.ones((4500,1,64,64)))
c = np.arange(4500, dtype = np.float64)
c

In [ ]:
def wrapper_ret():
    @tfe.defun
    def testfun(inp):
        #print(inp[1])
        return 0*tf.reduce_sum(inp[0]) + tf.reduce_sum(inp[1])
    @tfe.defun
    def testfun_wrap(tensors,coeffs):
        return tf.map_fn(testfun, (tf.split(tensors, 3), tf.split(c,3)),dtype = tf.float64)
    return testfun_wrap

In [ ]:
f = wrapper_ret()
#testfun_wrap(testdata,c)
import time
t0 = time.time()
f(testdata,c)
t1 = time.time()
print(t1-t0)
#tf.split(testdata, 3)

In [ ]:
q = np.array([[0,2],[1,3]], dtype = np.int32)
a = tf.random.uniform((6,4,5,5),dtype=tf.float32)
print(tf.gather_nd(a[0,0],q))
# print(a[0,2,...])
# print(a[1,3,...])
print(a[0,0,0,2])
print(a[0,0,1,3])
print(q.shape)

In [ ]:
q = tf.tile(tf.constant(np.array([[1,2,3], [2,3,4]]), dtype=tf.float32),[2,1])
print(q.shape)
s = tf.ones((12,3), dtype=tf.float32)
def f(x):
    return tf.reduce_sum(tf.einsum('ij,j->i', x[0], x[1]))
tf.map_fn(f, (tf.stack(tf.split(s,4)), q), dtype=tf.float32)

In [111]:
X,Y = np.meshgrid(np.linspace(0,1,110), np.linspace(0,2,73), indexing = 'xy')
f = tf.expand_dims(np.array([X + Y,X*Y, X-Y, X**2 + Y**2, X**2 - Y**2, X-2*Y]), axis = 1)

In [112]:
g = Lp_integral_norm((110,73), [0,1,0,2], n_quadpts=10, quadpts_randomization=1, p=2)

InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [110] != values[1].shape = [73] [Op:Pack] name: stack

In [110]:
g(f, np.zeros((6,1,110,73), dtype = np.float64))

<tf.Tensor: id=1083873, shape=(6,), dtype=float64, numpy=
array([2.30940114, 0.9428091 , 1.15470052, 2.92883149, 2.24106334,
       2.70801279])>

In [31]:
timeit g(tf.random.uniform((3000,1,64,64), dtype=tf.float64), tf.random.uniform((3000,1,64,64), dtype = tf.float64))

8.47 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
